# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)


# Review the features DataFrame
print(X.head())

   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = X_train.shape[1]

# Review the number of features
print("Number of input features:", number_input_features)

Number of input features: 11


In [ ]:
# Define the number of hidden nodes for the first hidden layer
number_output_neurons = 1

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer1 = 6
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
number_output_neurons = 1

In [ ]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [ ]:
# Display the Sequential model summary
print(nn.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 72        
                                                                 
 dense_4 (Dense)             (None, 3)                 21        
                                                                 
 dense_5 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model using 50 epochs and the training data
epochs = 50
fit_model = nn.fit(X_train_scaled, y_train, epochs=epochs, validation_split=0.2, verbose=1)

Epoch 1/50
30/30 [==============================] - 3s 19ms/step - loss: 0.6763 - accuracy: 0.5266 - val_loss: 0.6774 - val_accuracy: 0.5292
Epoch 2/50
30/30 [==============================] - 0s 5ms/step - loss: 0.6612 - accuracy: 0.5683 - val_loss: 0.6648 - val_accuracy: 0.6042
Epoch 3/50
30/30 [==============================] - 0s 6ms/step - loss: 0.6486 - accuracy: 0.6048 - val_loss: 0.6554 - val_accuracy: 0.6583
Epoch 4/50
30/30 [==============================] - 0s 6ms/step - loss: 0.6387 - accuracy: 0.6559 - val_loss: 0.6483 - val_accuracy: 0.6833
Epoch 5/50
30/30 [==============================] - 0s 8ms/step - loss: 0.6306 - accuracy: 0.6882 - val_loss: 0.6424 - val_accuracy: 0.6875
Epoch 6/50
30/30 [==============================] - 0s 7ms/step - loss: 0.6238 - accuracy: 0.6986 - val_loss: 0.6374 - val_accuracy: 0.7000
Epoch 7/50
30/30 [==============================] - 0s 6ms/step - loss: 0.6180 - accuracy: 0.7101 - val_loss: 0.6331 - val_accuracy: 0.7125
Epoch 8/50
30/30 [=

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=0)

# Display the model loss and accuracy results
print(f"Loss: {model_loss:.4f}")
print(f"Accuracy: {model_accuracy:.4f}")

Loss: 0.5126
Accuracy: 0.7425


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
model_file_path = "student_loans.keras"

# Export your model to a keras file
nn.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
model_file_path = "student_loans.keras"

# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(predictions[:5])

13/13 [==============================] - 0s 3ms/step
[[0.58720416]
 [0.33376864]
 [0.8556904 ]
 [0.6407556 ]
 [0.9241717 ]]


In [ ]:
import numpy as np

# Save the predictions to a DataFrame and round the predictions to binary results
binary_predictions = np.round(predictions).astype(int)
print(binary_predictions[:5])

[[1]
 [0]
 [1]
 [1]
 [1]]


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, binary_predictions))

              precision    recall  f1-score   support

           0       0.73      0.72      0.73       188
           1       0.76      0.76      0.76       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

Student academic information: GPA, major, expected graduation date, standardized test scores
Financial information: family income, savings, other financial aid, expected future earnings
Loan details: interest rates, repayment terms, loan amounts, eligibility criteria
Historical data: repayment success rates for similar student profiles
Career prospects: job market trends for different majors

This data is relevant because it helps assess the student's ability to repay the loan and match them with appropriate loan options. Academic information and career prospects indicate potential future earnings, while financial information shows current need and repayment capacity.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Content-based: Recommending loans based on the student's profile (major, GPA, financial situation) and matching it with loan attributes.
Context-based: Considering external factors like job market trends for the student's field of study and overall economic conditions.

This combination allows for personalized recommendations based on individual student characteristics while also accounting for broader economic factors that could affect loan repayment ability.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

a. Data privacy and security: Handling sensitive financial and personal information requires strict security measures and compliance with regulations like FERPA, GDPR, or CCPA. This is crucial because mishandling this data could lead to identity theft or financial fraud.
b. Ethical considerations and bias mitigation: Ensuring the system doesn't discriminate based on protected characteristics (race, gender, etc.) and provides equal opportunities for all students. This is important to avoid perpetuating or exacerbating existing inequalities in access to higher education and financial resources.
These challenges are particularly important for a student loan recommendation system because of the long-term impact loans can have on students' financial futures and the need to ensure fair access to educational opportunities.
